In [89]:
!pip install tensorflow
!pip install nltk
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [90]:
import tensorflow as tf

In [91]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [92]:
cd /content/drive/MyDrive/IndexPage

/content/drive/MyDrive/IndexPage


In [93]:
# new_model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/ModelLSDM')
# new_model.summary()

In [94]:
import pandas as pd 
import datetime
from tqdm.notebook import tqdm
import snscrape.modules.twitter as sntwitter

In [95]:
!pip install pyngrok
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [96]:
from pyngrok import ngrok
ngrok.set_auth_token('2NpJgVAPvnKxciFH5fjre1ZkRLl_6sr3HQirtTuoAciZZeoY3')

# Now you can use ngrok.connect() to expose a local web server


In [97]:
#cd /content/drive/MyDrive/ModelLSTM

In [98]:
import re

In [99]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [100]:
# removing not and no from stop words as we need them for our model training
stopwords = stopwords.words("english")
stopwords.remove('not')
stopwords.remove('no')

# intializing method for lemmatizing words
lemmatizer = WordNetLemmatizer()

# now creating funtion to clean our data
def cleaned_review(review):
    # remove any html tags
    new_review = BeautifulSoup(review).get_text()
    
    # remove urls from reviews
    no_urls = new_review.replace('http\S+', '').replace('www\S+', '')
    
    # remove any non-letters
    clean_review = re.sub("[^a-zA-Z]", " ", no_urls)
    
    # convert whole sentence to lowercase and split
    new_words = clean_review.lower().split()
    
    # converting stopwords list to set for faster search
    stops = set(stopwords)
    
    # using stopwords to remove irrelavent words and lemmatizing the final output
    final_words = [lemmatizer.lemmatize(word) for word in new_words if not word in stops]
    
    # return the final result
    return (" ".join(final_words))

In [101]:
# Importing required libraries
from tensorflow.keras.models import load_model
import numpy as np

# Loading the saved model
newModel = load_model('ModelLSDM')
newModel.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 50)            1154600   
                                                                 
 spatial_dropout1d (SpatialD  (None, 40, 50)           0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense (Dense)               (None, 50)                5050      
                                                                 
 dense_1 (Dense)             (None, 25)                1275      
                                                                 
 dense_2 (Dense)             (None, 3)                 78        
                                                        

In [102]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request

app = Flask(__name__)
run_with_ngrok(app)
import os

@app.route('/')
def index():

    return render_template("index.html")



In [103]:
# input_text = "fddf"

In [104]:
@app.route('/results', methods=['POST'])
def results():

    profile = request.form['username']
    print(profile)

    # Scrape Twitter profile
    now = datetime.datetime.now()
    twenty_four_hours_ago = now - datetime.timedelta(hours=24)
    formatted_time = twenty_four_hours_ago.strftime('%Y-%m-%d')
    userScraper = sntwitter.TwitterProfileScraper(profile)
    userdata = userScraper._get_entity()

    
    tweets = []
    input_text=""
    for i, tweet in enumerate(userScraper.get_items()):
        if isinstance(tweet, sntwitter.Tweet) and tweet.date.strftime('%Y-%m-%d') >= formatted_time:
            if(i > 1):
                break
            data = [tweet.date,
                    tweet.rawContent,
                    tweet.likeCount,
                    tweet.retweetCount]
            input_text += tweet.rawContent
            tweets.append(data)

    input_text = cleaned_review(input_text)

    # So we see above that max no.of words in each sentence is 40
    max_len = 40

    from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing.sequence import pad_sequences

    tokenizer = Tokenizer(num_words=10000)
    tokenizer.fit_on_texts(input_text)

    # # Preprocess input text
    input_sequence = tokenizer.texts_to_sequences([input_text])
    padded_input_sequence = pad_sequences(input_sequence, maxlen=max_len, padding='post')
    
    # Make prediction
    preds = newModel.predict(padded_input_sequence)
    sentiment_labels = ['Negative', 'Neutral', 'Positive']
    predicted_label = sentiment_labels[np.argmax(preds)]


    return render_template('results.html', predicted_label=predicted_label)


In [ ]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://b94b-35-237-13-170.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [04/Apr/2023 07:04:55] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Apr/2023 07:04:55] "GET /static/js/script.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Apr/2023 07:04:55] "GET /static/css/index.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Apr/2023 07:04:56] "GET /static/js/trending.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Apr/2023 07:04:56] "GET /static/js/search.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Apr/2023 07:04:56] "GET /static/js/top100.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Apr/2023 07:04:56] "GET /static/images/Logo.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Apr/2023 07:04:56] "GET /static/images/carousel-1.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Apr/2023 07:04:56] "GET /static/images/carousel-2.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Apr/2023 07:04:57] "GET /static/images/background.jpg HTTP/1.1" 200 -
